In [ ]:
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import ArxivLoader

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()  # Loads .env file into environment
api_key = os.getenv("OPENAI_API_KEY")

print("✅ API key loaded:", bool(api_key))

True

In [ ]:
# Initialize models
orchestrator = ChatOpenAI(model="gpt-4o-mini", temperature=0)
plan_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
draft_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
critic_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [ ]:
search_tool = DuckDuckGoSearchRun()
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectordb = Chroma(persist_directory="./vectorstore", embedding_function=embeddings)

In [ ]:
def route_query(query):
    """Classify query as general or research."""
    prompt = f"Classify this query as 'general' or 'research': {query}"
    result = orchestrator.invoke(prompt)
    return result.content.strip().lower()

def general_answer(query):
    return plan_llm.invoke(query).content

def analyzer_collect(query):
    """Retrieve context using web search + Chroma."""
    web_results = search_tool.run(query)
    vectordb.add_texts([web_results])
    retriever = vectordb.as_retriever(search_kwargs={"k": 3})
    context_docs = retriever.invoke(query)
    return "\n\n".join([d.page_content for d in context_docs])

def plan_writer(query, context):
    plan_prompt = ChatPromptTemplate.from_template("""
    Based on the user's goal:
    {query}

    And this context:
    {context}

    Write a clear, step-by-step plan.
    """)
    return plan_llm.invoke(plan_prompt.format(query=query, context=context)).content

def draft_writer(plan):
    return draft_llm.invoke(f"Write a structured essay based on this plan:\n{plan}").content

def critic_agent(draft):
    return critic_llm.invoke(f"Critique this draft and suggest improvements:\n{draft}").content

def final_drafter(text):
    final_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
    return final_llm.invoke(f"Polish and finalize this essay:\n{text}").content

def orchestrate_agent(query):
    intent = route_query(query)
    if intent == "general":
        return general_answer(query)
    else:
        context = analyzer_collect(query)
        plan = plan_writer(query, context)
        draft = draft_writer(plan)
        critique = critic_agent(draft)
        print("🧠 Critique:", critique)
        final = final_drafter(draft)
        vectordb.add_texts([final])
        return final

In [ ]:
# --- agent_graph.py ---

# 1️⃣ Imports
from langgraph.graph import StateGraph, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# 2️⃣ Define global tools / LLMs
orchestrator = ChatOpenAI(model="gpt-4o-mini", temperature=0)
plan_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
draft_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
critic_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

search_tool = DuckDuckGoSearchRun()
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectordb = Chroma(persist_directory="./notebooks/vectorstore", embedding_function=embeddings)

# 3️⃣ Define state (memory structure)
class AgentState(TypedDict):
    query: str
    intent: str
    context: str
    plan: str
    draft: str
    critique: str
    final: str
    research_mode: str 

# 4️⃣ Define all agent functions (logic)
def route_query(state: dict):
    """Classify query as general or research and update state."""
    query = state.get("query", "")
    prompt = f"Classify this query as 'general' or 'research': {query}"
    result = orchestrator.invoke(prompt)
    state["intent"] = result.content.strip().lower()
    print(f"🧭 Intent classified as: {state['intent']}")
    return state


def is_vague_query(query: str) -> bool:
    """Simple heuristic to detect vague research questions."""
    vague_terms = ["research", "impact", "effect", "study", "analyze", "discuss", "AI", "technology"]
    return len(query.split()) < 5 or any(term in query.lower() for term in vague_terms)


def general_answer(state: AgentState):
    """Handle general questions not requiring research."""
    query = state.get("query", "")
    result = plan_llm.invoke(query)
    state["final"] = result.content
    print("💬 General answer generated.")
    return state


def analyzer_collect(state: AgentState):
    """Retrieve and embed context using web search + Chroma DB."""
    query = state.get("query", "")
    print("🔍 Searching the web for context...")
    web_results = search_tool.run(query)

    # Add search results to vector store
    vectordb.add_texts([web_results])

    # Retrieve top 3 relevant chunks
    retriever = vectordb.as_retriever(search_kwargs={"k": 3})
    context_docs = retriever.invoke(query)

    # Join the document contents for later context
    state["context"] = "\n\n".join([d.page_content for d in context_docs])
    print("📚 Context collected.")
    return state

def arxiv_collect(state: AgentState):
    """Load academic papers from arXiv related to the query."""
    query = state.get("query", "")
    print("📖 Searching arXiv for relevant papers...")

    try:
        loader = ArxivLoader(query=query, load_max_docs=3)
        docs = loader.load()
        contents = [doc.page_content for doc in docs if doc.page_content]
        titles = [doc.metadata.get("Title", "Untitled") for doc in docs]

        # Save context in state
        state["context"] = "\n\n".join(contents)
        state["final"] = "🧾 Retrieved arXiv papers:\n" + "\n".join([f"• {t}" for t in titles])
        print(f"✅ Loaded {len(docs)} papers from arXiv.")
    except Exception as e:
        state["context"] = ""
        state["final"] = f"⚠️ ArxivLoader error: {e}"
        print(state["final"])
    
    return state

def decide_research_mode(state: AgentState):
    """Decide whether the user needs document search or writing assistance."""
    query = state.get("query", "")
    prompt = f"""
    The user asked: "{query}"

    Decide the intent:
    - If they just want information or sources → output 'search'
    - If they need to write an essay or academic answer → output 'write'
    """
    result = orchestrator.invoke(prompt)
    mode = result.content.strip().lower()
    print(f"🧭 Research mode detected: {mode}")
    state["research_mode"] = mode
    return state


def list_documents(state: AgentState):
    """Return a list of the most relevant document summaries."""
    query = state.get("query", "")
    retriever = vectordb.as_retriever(search_kwargs={"k": 5})
    docs = retriever.invoke(query)

    summaries = "\n\n".join([f"📄 {i+1}. {d.page_content[:300]}..." for i, d in enumerate(docs)])
    state["final"] = f"Top 5 relevant documents:\n\n{summaries}"
    print("📄 Generated top 5 relevant document list.")
    return state

def plan_writer(state: AgentState):
    """Generate an outline or plan for the essay/answer."""
    query = state.get("query", "")
    context = state.get("context", "")

    plan_prompt = ChatPromptTemplate.from_template("""
    Based on the user's goal:
    {query}

    And this context:
    {context}

    Write a clear, step-by-step plan for producing an academic-style answer.
    """)

    result = plan_llm.invoke(plan_prompt.format(query=query, context=context))
    state["plan"] = result.content
    print("📝 Plan written.")
    return state


def draft_writer(state: AgentState):
    """Generate the main draft based on the plan."""
    plan = state.get("plan", "")
    draft = draft_llm.invoke(f"Write a detailed essay based on this plan:\n{plan}")
    state["draft"] = draft.content
    print("✍️ Draft generated.")
    return state


def critic_agent(state: AgentState):
    """Critique and suggest improvements for the draft."""
    draft = state.get("draft", "")
    critique = critic_llm.invoke(f"Critique this essay and suggest specific improvements:\n{draft}")
    state["critique"] = critique.content
    print("🧾 Critique complete.")
    return state


def final_drafter(state: AgentState):
    """Polish and finalize the essay, then store in vector DB."""
    draft = state.get("draft", "")
    final_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
    final = final_llm.invoke(f"Polish and finalize this essay:\n{draft}")

    # Save final answer for retrieval
    vectordb.add_texts([final.content])
    vectordb.persist()

    state["final"] = final.content
    print("✅ Final version saved.")
    return state

In [ ]:
# --- Graph Definition ---
graph = StateGraph(AgentState)

# Existing nodes
graph.add_node("route_query", route_query)
graph.add_node("general_answer", general_answer)
graph.add_node("analyzer_collect", analyzer_collect)
graph.add_node("arxiv_collect", arxiv_collect)
graph.add_node("plan_writer", plan_writer)
graph.add_node("draft_writer", draft_writer)
graph.add_node("critic_agent", critic_agent)
graph.add_node("final_drafter", final_drafter)

# 🆕 New nodes
graph.add_node("decide_research_mode", decide_research_mode)
graph.add_node("list_documents", list_documents)

# --- Conditional branching from route_query ---
def route_decision(state):
    if state["intent"] == "general":
        return "general_answer"
    else:
        query = state.get("query", "")
        if is_vague_query(query):
            return "arxiv_collect"
        else:
            return "analyzer_collect"


graph.add_conditional_edges(
    "route_query",
    route_decision,
    {
        "general_answer": "general_answer",
        "arxiv_collect": "arxiv_collect",
        "analyzer_collect": "analyzer_collect"
    }
)

# --- New conditional branching from analyzer_collect ---
def research_decision(state):
    if state.get("research_mode") == "search":
        return "list_documents"
    else:
        return "plan_writer"


graph.add_edge("analyzer_collect", "decide_research_mode")
graph.add_edge("arxiv_collect", "decide_research_mode")

graph.add_conditional_edges(
    "decide_research_mode",
    research_decision,
    {
        "list_documents": "list_documents",
        "plan_writer": "plan_writer"
    }
)

# Linear flow for writing
graph.add_edge("plan_writer", "draft_writer")
graph.add_edge("draft_writer", "critic_agent")
graph.add_edge("critic_agent", "final_drafter")

# End points
graph.add_edge("final_drafter", END)
graph.add_edge("list_documents", END)

# Entry point
graph.set_entry_point("route_query")

# Compile graph
app = graph.compile()
app


In [ ]:
state = {"query": "Who developed the first generative AI model?"}
final_state = app.invoke(state)
print("\n--- RESULT ---")
print(final_state["final"])

In [ ]:
# 2️⃣ Vague academic search → arXiv
state = {"query": "5 Researches about GenAI and Agentic AI"}
final_state = app.invoke(state)
print("\n--- RESULT ---")
print(final_state["final"])